In [72]:
from pandas import read_csv 
from pandas import concat
from datetime import datetime
import csv
import time 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
# load all data
cols = list(read_csv("datatest.csv", nrows =1))
data1 = read_csv('datatest.csv', usecols = [i for i in cols if i != 'No'])
data2 = read_csv('datatraining.csv', usecols = [i for i in cols if i != 'No'])
data3 = read_csv('datatest2.csv', usecols = [i for i in cols if i != 'No'])
# vertically stack and maintain temporal order
data = concat([data1, data2, data3])
data.to_csv('combined.csv')
data.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
1,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
2,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
3,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
4,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [73]:
f = open('combined.csv', 'r')
new_f = open('processed.csv', "a")

data = csv.reader(f)
new_data = csv.writer(new_f, delimiter=',')

new_row = ['Month', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']
new_data.writerow(new_row)

first = 0
for row in data:
# skip the first row 
    if first == 0:
        first = 1
        continue

    id = row[0]
    datetime_str  = row[1]

    # "1","2015-02-04 17:51:00",23.18,27.272,426,721.25,0.00479298817650529,1

    dt_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    hour =  dt_obj.strftime("%d")
    month = dt_obj.strftime("%m")

    new_row = [month, hour, row[2], row[3], row[4], row[5], row[6], row[7]]
    new_data.writerow(new_row)
 
f.close()
new_f.close()

In [74]:
data = read_csv('processed.csv')
data.head()

,Month,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2,2,23.7000,26.272,585.200000,749.200000,0.004764,1
1,2,2,23.7180,26.290,578.400000,760.400000,0.004773,1
2,2,2,23.7300,26.230,572.666667,769.666667,0.004765,1
3,2,2,23.7225,26.125,493.750000,774.750000,0.004744,1
4,2,2,23.7540,26.200,488.600000,779.000000,0.004767,1


In [77]:
# logistic regression feature selection
# load the dataset
data = read_csv('processed.csv')
values = data.values
# basic feature selection
features = [0, 1, 2, 3, 4, 5, 6]
for f in features:
	# split data into inputs and outputs
	X, y = values[:, f].reshape((len(values), 1)), values[:, -1]
	# split the dataset
	trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=1)
	# define the model
	model = LogisticRegression()
	# fit the model on the training set
	model.fit(trainX, trainy)
	# predict the test set
	yhat = model.predict(testX)
	# evaluate model skill
	score = accuracy_score(testy, yhat)
	print('feature=%d, name=%s, score=%.3f' % (f, data.columns[f], score))

feature=0, name=Month, score=0.822
feature=1, name=Hour, score=0.822
feature=2, name=Temperature, score=0.799
feature=3, name=Humidity, score=0.822
feature=4, name=Light, score=0.991
feature=5, name=CO2, score=0.763
feature=6, name=HumidityRatio, score=0.822
